In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip uninstall transformers torch accelerate -y
!pip install transformers[torch] accelerate

Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: torch 2.3.0+cu121
Uninstalling torch-2.3.0+cu121:
  Successfully uninstalled torch-2.3.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=3ab838b19a8ce27d91254401b3e5e66c6655057c583b3096630a628ae6de8ab3
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset,Audio

dataset = load_dataset("csv",data_files="/content/drive/MyDrive/datos_finale.csv",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['label', 'audio'],
    num_rows: 2116
})

In [ ]:
dataset = dataset.remove_columns(["ID_frase", "conv","speaker","video_name","emotion_x","text"])
dataset=dataset.rename_column('emocion_code','label')

In [ ]:
from datasets import load_dataset, DatasetDict
import numpy as np
import ast
# Definir una función de procesamiento para convertir las cadenas en arrays de numpy
def convert_to_numpy(example):
    example['audio'] = np.array(ast.literal_eval(example['audio']))
    return example

# Aplicar la función de procesamiento al dataset



In [ ]:
processed_dataset = dataset.map(convert_to_numpy)

Map:   0%|          | 0/2116 [00:00<?, ? examples/s]

In [ ]:
train_testvalid = processed_dataset.train_test_split(test_size=0.2)
test = train_testvalid['test']

train_valid = train_testvalid['train'].train_test_split(test_size=0.25)
train = train_valid['train']
val = train_valid['test']

In [ ]:
from transformers import  Wav2Vec2FeatureExtractor, AutoConfig

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    audio_arrays = examples["audio"]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [ ]:
encoded_minds_test = test.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds_train = train.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds_val = val.map(preprocess_function, remove_columns="audio", batched=True)


Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Map:   0%|          | 0/1269 [00:00<?, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

In [ ]:
encoded_minds

Dataset({
    features: ['label', 'input_values'],
    num_rows: 2116
})

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = 6
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/hubert-base-ls960", num_labels=num_labels
)

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
training_args = TrainingArguments(
    output_dir="hubert_audio",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=15,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds_train,
    eval_dataset=encoded_minds_test,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.757100,1.708145,0.240566
2,1.667900,1.599322,0.323113
3,1.608500,1.584034,0.323113
4,1.597500,1.578111,0.323113
5,1.594400,1.576795,0.323113
6,1.593100,1.575986,0.323113
7,1.595700,1.576954,0.323113
8,1.591200,1.579074,0.323113
9,1.586400,1.576785,0.323113
10,1.591700,1.574306,0.323113


TrainOutput(global_step=150, training_loss=1.608733558654785, metrics={'train_runtime': 621.4245, 'train_samples_per_second': 30.631, 'train_steps_per_second': 0.241, 'total_flos': 1.7281365082848e+17, 'train_loss': 1.608733558654785, 'epoch': 15.0})

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("drigoro/my_awesome_audio_model")
inputs = feature_extractor(array_vfi, sampling_rate=16000, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
task_evaluator = evaluator("audio-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline="Drigoro/hubert_audio",
    data=encoded_minds_val,
    metric=evaluate.combine(["accuracy"]),
    label_mapping={"ANGER": 0, "DISGUST": 1,"FEAR":2,"JOY":3,"SADNESS":4,"SURPRISE":5}
)
print(eval_results)